In [ ]:
from flask import Flask, render_template, request, redirect
from pymongo import MongoClient

app = Flask(__name__)

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['survey_db']
collection = db['user_data']

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    # Get data from the form
    user_data = {
        "age": request.form.get('age'),
        "gender": request.form.get('gender'),
        "total_income": request.form.get('income'),
        "expenses": {
            "utilities": request.form.get('utilities'),
            "entertainment": request.form.get('entertainment'),
            "school_fees": request.form.get('school_fees'),
            "shopping": request.form.get('shopping'),
            "healthcare": request.form.get('healthcare')
        }
    }
    
    # Store data in MongoDB
    collection.insert_one(user_data)
    
    return redirect('/')

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Survey Form</title>
</head>
<body>
    <h1>Income Spending Survey</h1>
    <form action="/submit" method="POST">
        Age: <input type="text" name="age"><br>
        Gender: <input type="text" name="gender"><br>
        Total Income: <input type="text" name="income"><br><br>
        <h3>Expenses</h3>
        Utilities: <input type="text" name="utilities"><br>
        Entertainment: <input type="text" name="entertainment"><br>
        School Fees: <input type="text" name="school_fees"><br>
        Shopping: <input type="text" name="shopping"><br>
        Healthcare: <input type="text" name="healthcare"><br><br>
        <button type="submit">Submit</button>
    </form>
</body>
</html>


In [ ]:
import csv
from pymongo import MongoClient

class User:
    def __init__(self):
        self.client = MongoClient("mongodb://localhost:27017/")
        self.db = self.client['survey_db']
        self.collection = self.db['user_data']

    def save_to_csv(self, filename="user_data.csv"):
        data = self.collection.find()
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['age', 'gender', 'total_income', 'utilities', 'entertainment', 'school_fees', 'shopping', 'healthcare']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for item in data:
                row = {
                    'age': item['age'],
                    'gender': item['gender'],
                    'total_income': item['total_income'],
                    'utilities': item['expenses']['utilities'],
                    'entertainment': item['expenses']['entertainment'],
                    'school_fees': item['expenses']['school_fees'],
                    'shopping': item['expenses']['shopping'],
                    'healthcare': item['expenses']['healthcare']
                }
                writer.writerow(row)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data from CSV
df = pd.read_csv('user_data.csv')

# Show the ages with the highest income
plt.figure(figsize=(10, 5))
df.groupby('age')['total_income'].sum().plot(kind='bar')
plt.title('Total Income by Age')
plt.xlabel('Age')
plt.ylabel('Total Income')
plt.show()

# Show the gender distribution across spending categories
expense_columns = ['utilities', 'entertainment', 'school_fees', 'shopping', 'healthcare']
df.groupby('gender')[expense_columns].sum().plot(kind='bar', stacked=True)
plt.title('Spending by Gender')
plt.xlabel('Gender')
plt.ylabel('Total Spending')
plt.show()

# Export the charts
plt.savefig('spending_by_gender.png')


In [ ]:
from waitress import serve

if __name__ == "__main__":
    serve(app, host='0.0.0.0', port=8080)
